In [1]:
!pip install tensorflow_ranking tensorflow_probability tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 78.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 60.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing inst

In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, LSTM, Permute, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Reshape, SpatialDropout1D, Dropout, TimeDistributed

from keras.regularizers import l2, l1
from keras.optimizers import Adam

import sklearn
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import pickle

import VAE
from VAE import Sampling, VAE, get_auth_model_from_latent_space, get_decoder, get_encoder, get_auth_model
from VAE_stats import VAE_stats
from scaler import CustomScaler


import datetime, csv, os, re, sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, f1_score, precision_score, precision_recall_curve, recall_score, roc_curve
from sklearn.model_selection import StratifiedKFold

from scaler import CustomScaler

from featurize import filter, featurize
from sklearn.neural_network import MLPClassifier

#configs
maxprewindowsize = 4
classifier = 'rfc'

folds = 10


def get_average(l):
	return 0 if 0 == len(l) else sum(l) / len(l)

def get_eer(scores_legit, scores_adv):
	scores_legit = sorted(scores_legit)
	scores_adv = sorted(scores_adv)
	
	#treat each legitimate sample distance as a possible threshold, determine the point where FRR crosses FAR
	for c, threshold in enumerate(scores_legit):
		frr = c * 1.0 / len(scores_legit)
		adv_index = next((x[0] for x in enumerate(scores_adv) if x[1] > threshold), len(scores_adv))
		far = 1 - (adv_index * 1.0 / len(scores_adv))
		if frr >= far:
			return threshold, far
	print("Failure")

def get_eer_recogblind(scores_legit, scores_adv_typed, total_w, total_b, total_i):
	scores_legit = sorted(scores_legit)
	scores_adv_typed = sorted(scores_adv_typed, key = lambda x:x[0])
	
	#treat each legitimate sample distance as a possible threshold, determine the point where FRR crosses FAR
	for c, threshold in enumerate(scores_legit):
		frr = c * 1.0 / len(scores_legit)
		adv_index = next((x[0] for x in enumerate(scores_adv_typed) if x[1][0] > threshold), len(scores_adv_typed))
		far = 1 - (adv_index * 1.0 / len(scores_adv))
		if frr >= far:
			rejectrate_w = 0 if 0 == total_w else len([i for i in scores_adv_typed if 'W' == i[1] and i[0] >= threshold]) / total_w
			rejectrate_b = 0 if 0 == total_b else len([i for i in scores_adv_typed if 'B' == i[1] and i[0] >= threshold]) / total_b
			rejectrate_i = 0 if 0 == total_i else len([i for i in scores_adv_typed if 'I' == i[1] and i[0] >= threshold]) / total_i
			return threshold, far, rejectrate_w, rejectrate_b, rejectrate_i

def get_far_when_zero_frr(scores_legit, scores_adv):
	scores_legit = sorted(scores_legit)
	scores_adv = sorted(scores_adv)
	
	#treat each legitimate sample distance as a possible threshold, determine the point with the lowest FAR that satisfies the condition that FRR = 0
	for c, threshold in enumerate(scores_legit):
		frr = c * 1.0 / len(scores_legit)
		adv_index = next((x[0] for x in enumerate(scores_adv) if x[1] > threshold), len(scores_adv))
		far = 1 - (adv_index * 1.0 / len(scores_adv))
		if frr > 0.001:
			return threshold, far

def plot_threshold_by_far_frr(scores_legit, scores_adv, far_theta):
	scores_legit = sorted(scores_legit)
	scores_adv = sorted(scores_adv)
	
	frr = []
	far = []
	thresholds = []
	for c, threshold in enumerate(scores_legit):
		frr.append((c * 1.0 / len(scores_legit)) * 100)
		adv_index = next((x[0] for x in enumerate(scores_adv) if x[1] > threshold), len(scores_adv))
		far.append((1 - (adv_index * 1.0 / len(scores_adv))) * 100)
		thresholds.append(threshold)
	plt.figure(figsize = (6, 6))
	#plt.rcParams.update({'font.size': fontsize_legends})
	plt.plot(thresholds, far, 'tab:blue', label = 'FAR')
	plt.plot(thresholds, frr, 'tab:orange', label = 'FRR')
	plt.ylabel('error rate (%)')
	plt.xlabel(r'decision threshold, $\theta$')
	plt.axvline(x = far_theta, c = 'red')
	plt.legend(loc = 'best')
	plt.tight_layout(pad = 0.05)
	plt.show()

def plot_threshold_by_precision_recall(labels_test, labels_scores):
	p, r, thresholds = precision_recall_curve(labels_test, labels_scores)
	plt.figure(figsize = (6, 6))
	#plt.rcParams.update({'font.size': fontsize_legends})
	plt.title('Precision and Recall Scores as a Function of the Decision Threshold', fontsize = 12)
	plt.plot(thresholds, p[:-1], 'tab:blue', label = 'precision')
	plt.plot(thresholds, r[:-1], 'tab:orange', label = 'recall')
	plt.ylabel('score')
	plt.xlabel(r'decision threshold, $\theta$')
	plt.legend(loc = 'best')
	plt.tight_layout(pad = 0.05)
	plt.show()

def plot_roc_curve(labels_test, labels_scores):
	fpr, tpr, auc_thresholds = roc_curve(labels_test, labels_scores)
	print('AUC of ROC = ' + str(auc(fpr, tpr)))
	plt.figure(figsize = (6, 6))
	#plt.rcParams.update({'font.size': fontsize_legends})
	plt.title('ROC Curve', fontsize = 12)
	plt.plot(fpr, tpr, 'tab:orange', label = 'recall optimized')
	plt.plot([0, 1], [0, 1], 'k--')
	plt.axis([-0.005, 1, 0, 1.005])
	plt.xticks(np.arange(0, 1, 0.05), rotation = 90)
	plt.xlabel('false positive rate')
	plt.ylabel('true positive rate (recall)')
	plt.legend(loc = 'best')
	plt.tight_layout(pad = 0.05)
	plt.show()

def get_ascending_userID_list_string():
	for u in userIDs:
		if not 'user' in u and len(u) != 7:
			sys.exit('ERROR: userID not valid: ' + str(u))
	IDs = [int(u[4:]) for u in userIDs]
	IDs.sort(reverse = False)
	return ','.join([f'{i:03}' for i in IDs])

def get_descending_feature_list_string(weights, labels, truncate = 0):
	indicies = [i for i in range(len(weights))]
	for i in range(len(indicies)):
		for j in range(len(indicies)):
			if i != j and weights[indicies[i]] > weights[indicies[j]]:
				temp = indicies[i]
				indicies[i] = indicies[j]
				indicies[j] = temp
	if truncate != 0:
		del indicies[truncate:]
	return '\n'.join([str('%.6f' % weights[i]) + ' (' + labels[i] + ')' for i in indicies])

def write_verbose(f, s):
	outfilename = f + '-verbose.txt'
	outfile = open(outfilename, 'a')
	outfile.write(s + '\n')
	outfile.close()
    

class SplitLayer(tf.keras.layers.Layer):
    def __init__(self, layers):
        super(SplitLayer, self).__init__()
        self.layers = layers

    def call(self, inputs):
        return tf.gather(inputs,indices=self.layers,axis=-1)


def get_new_auth_model(input_dim=(200,16)):
    #input_dim = (200,16)
    
    inputs = keras.Input(shape=input_dim)
    x = inputs

    xs = []
    
    for i in range(16):
        
        x = SplitLayer(i)(inputs)
        reshaped = Reshape((200, 1))(x)
        
        x = Conv1D(100, 10, strides=2, padding="same")(reshaped)
        x = MaxPooling1D(pool_size=2, strides=None, padding="same")(x)
        #x = SpatialDropout1D(0.2)(x)
        
        x = Conv1D(100, 3, strides=1, padding="same")(x)
        x = MaxPooling1D(pool_size=2, strides=None, padding="same")(x)
        #x = SpatialDropout1D(0.2)(x)

        xs.append(x)
    
    x = layers.Concatenate()(xs)

    
    x = LSTM(50)(x)

    x = Dense(25, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(10, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    #x = layers.Lambda(lambda x: x / 2)
    out = x

    model = keras.Model(inputs, out, name="LSTM_classifier_v2")
    
    return model


def run(models=["WatchAuth"], forbidden_stat = 0, users = range(16), limit_data_first_x = None, median_filtering= False, terminal_types = [3,4,5,6,7,8,9], score_mode= "macro", extra_data_handle=None, repetitions=1, save_name = "recent_results"):
    
    file_name = "raw_with_maps" # or offsets_2
    vae_stats = VAE_stats(10)
    
    if median_filtering:
        x_data = np.load(f"data/processed/x_{file_name}_median_filtered.npy")
        feature_x_data = np.load(f"data/processed/x_{file_name}_median_filtered_features.npy")
    else:
        x_data = np.load(f"data/processed/x_{file_name}_filtered.npy")  # _filtered
        feature_x_data = np.load(f"data/processed/x_{file_name}_features.npy")

    x_data = x_data[:, :, [0,1,2,3,4,5,6,7]]
    feature_x_data = vae_stats.vae_featurize(x_data)
        
    y_user = np.load(f"data/processed/y_user_{file_name}.npy")
    y_intent = np.load(f"data/processed/y_intent_{file_name}.npy")
    y_gesture = np.load(f"data/processed/y_gesture_type_{file_name}.npy")

    train_gesture_map = np.load(f"data/processed/train_gesture_map_{file_name}.npy")
    test_gesture_map = np.load(f"data/processed/test_gesture_map_{file_name}.npy")
    
    feature_array = feature_x_data

    

    train_data_map = train_gesture_map.astype(bool)
    test_data_map = test_gesture_map.astype(bool)

    output = {}
    
    all_scores_legit, all_scores_adv = [], []
    
    for auth_user in users:
        print(f"auth_user is {auth_user}")
           
        
        zero_x_data = np.load(f"data/generated_samples/user={auth_user}_model=vae_gan_specific_50_gestures=2_non_user.npy")
        zero_x_data = zero_x_data[:, :, [0,1,2,3,4,5,6,7]]
        
        zero_x_data[:,:,3] = (zero_x_data[:,:,0]**2 + zero_x_data[:,:,1]**2 + zero_x_data[:,:,2]**2)**0.5
        zero_x_data[:,:,7] = (zero_x_data[:,:,4]**2 + zero_x_data[:,:,5]**2 + zero_x_data[:,:,6]**2)**0.5
            
        one_x_data = np.load(extra_data_handle(auth_user))
        one_x_data = one_x_data[:, :, [0,1,2,3,4,5,6,7]]
        
        one_x_data[:,:,3] = (one_x_data[:,:,0]**2 + one_x_data[:,:,1]**2 + one_x_data[:,:,2]**2)**0.5
        one_x_data[:,:,7] = (one_x_data[:,:,4]**2 + one_x_data[:,:,5]**2 + one_x_data[:,:,6]**2)**0.5
          

        if median_filtering:
            one_x_feature_data = np.zeros((len(one_x_data), feature_x_data.shape[1]))

            for i in range(0, len(one_x_data), 128):
                median_filtered_one_data = tfa.image.median_filter2d(one_x_data[i:i+128],filter_shape=(1,5), padding="constant")
                one_x_feature_data[i:i+128] = vae_stats.vae_featurize(median_filtered_one_data)
            
            zero_x_feature_data = np.zeros((len(zero_x_data), feature_x_data.shape[1]))

            for i in range(0, len(zero_x_data), 128):
                median_filtered_zero_data = tfa.image.median_filter2d(zero_x_data[i:i+128],filter_shape=(1,5), padding="constant")
                zero_x_feature_data[i:i+128] = vae_stats.vae_featurize(median_filtered_zero_data)
                

        data_train = np.concatenate([zero_x_data, one_x_data])
        feature_data_train = np.concatenate([zero_x_feature_data, one_x_feature_data])
        labels_train = np.concatenate([np.zeros(len(zero_x_data)), np.ones(len(one_x_data))])
        
        feature_data_test = feature_array[test_data_map]
        data_test = x_data[test_data_map]
        labels_test = (y_user.argmax(axis=1) == auth_user)[test_data_map].astype(int)
        
        #print(np.unique(labels_train, return_counts=True))
        
        for model_name in models:  # "DL", "MLP", "SVM", 

            key = str(auth_user) + "_" + model_name
            output[key] = {"fm":[], "prec":[], "rec":[], "eer":[], "far":[]}


            a_precisions = []
            a_recalls = []
            a_fmeasures = []
            a_pr_stdev = []
            a_re_stdev = []
            a_fm_stdev = []
            a_eers = []
            a_eer_thetas = []
            a_fars = []
            a_far_thetas = []
            a_ee_stdev = []
            a_ee_th_stdev = []
            a_fa_stdev = []
            a_fa_th_stdev = []
            a_aurocs = []
            a_losses = []


            
            if model_name in ["DL", "SVM"]:
                reps = 1
            else:
                reps = repetitions


            for repetition in range(reps):
                if model_name == "DL":

                    shuffled_data_train, shuffled_labels_train = shuffle(data_train, labels_train, random_state=0)

                    #ORDER = 6
                    #CUTOFF = 3.667
                    #shuffled_data_train = filter(shuffled_data_train, list(range(16)), ORDER, CUTOFF)

                    scaler = CustomScaler()
                    shuffled_data_train = scaler.fit_and_transform(shuffled_data_train)


                    kFold = sklearn.model_selection.StratifiedKFold(n_splits=5)
                    val_map = next(kFold.split(shuffled_data_train, shuffled_labels_train))[1]


                    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', start_from_epoch=50,
                                                                               patience=50, restore_best_weights=True)


                    model = None
                    model = get_new_auth_model((200,16))
                    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss="binary_crossentropy")
                    history = model.fit(shuffled_data_train[~val_map], shuffled_labels_train[~val_map], epochs=1000,
                                        batch_size=128, verbose=1,
                                        validation_data=(shuffled_data_train[val_map], shuffled_labels_train[val_map]),
                                       class_weight={0:0.5, 1:30},
                                       callbacks=[early_stopping_callback])

                    plt.plot(history.history['loss'])
                    plt.plot(history.history['val_loss'])
                    plt.title('model loss')
                    plt.ylabel('loss')
                    plt.xlabel('epoch')
                    plt.legend(['train', 'val'], loc='upper left')
                    plt.show()

                    plt.clf()

                    labels_scores = model.predict(scaler.transform(data_test)).squeeze()
                    labels_pred = (labels_scores >= 0.5).astype(int)



                elif model_name == "WatchAuth":
                    model = RandomForestClassifier(n_estimators = 100, random_state = repetition, class_weight="balanced").fit(feature_data_train, labels_train)


                    labels_pred = model.predict(feature_data_test)
                    labels_scores = model.predict_proba(feature_data_test)[:, 1]

                elif model_name == "SVM":

                    clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True, class_weight="balanced"))
                    clf.fit(feature_data_train, labels_train)

                    labels_pred = clf.predict(feature_data_test)
                    labels_scores = clf.predict_proba(feature_data_test)[:, 1]

                elif model_name == "MLP":

                    clf = make_pipeline(StandardScaler(), MLPClassifier(solver='lbfgs', alpha=1e-3,
                             hidden_layer_sizes=(50,20, 2), random_state=repetition)) # ,class_weight="balanced")) 
                    clf.fit(feature_data_train, labels_train)

                    labels_pred = clf.predict(feature_data_test)
                    labels_scores = clf.predict_proba(feature_data_test)[:, 1]

                #get precision, recall, and F-measure scores
                precision = precision_score(labels_test, labels_pred, average = score_mode, labels = np.unique(labels_pred))
                recall = recall_score(labels_test, labels_pred, average = score_mode, labels = np.unique(labels_pred))
                fmeasure = f1_score(labels_test, labels_pred, average = score_mode, labels = np.unique(labels_pred)) # macro vs binary
                auroc = sklearn.metrics.roc_auc_score(labels_test, labels_scores, average = "macro", labels = np.unique(labels_pred))
                a_precisions.append(precision)
                a_recalls.append(recall)
                a_fmeasures.append(fmeasure)
                a_aurocs.append(auroc)
                a_losses.append(sklearn.metrics.log_loss(labels_test, labels_scores))
                #print(fmeasure)

                #print(f"Loss is {sklearn.metrics.log_loss(labels_test, labels_scores)}")
                #print(precision)
                #print(recall)


                scores_legit = [labels_scores[i] for i in range(len(labels_test)) if 1 == labels_test[i]]
                scores_adv = [labels_scores[i] for i in range(len(labels_test)) if 0 == labels_test[i]]
                eer_theta, eer = get_eer(scores_legit, scores_adv)
                far_theta, far = get_far_when_zero_frr(scores_legit, scores_adv)

                all_scores_legit += scores_legit
                all_scores_adv += scores_adv
                #plot_roc_curve(labels_test, labels_scores)
                #plot_threshold_by_precision_recall(labels_test, labels_scores)
                #plot_threshold_by_far_frr(scores_legit, scores_adv, far_theta)

                #print(f"eer is {eer}")
                a_eers.append(eer)
                a_eer_thetas.append(eer_theta)
                a_fars.append(far)
                a_far_thetas.append(far_theta)
                
                



            a_pr_stdev = np.std(a_precisions, ddof = 1)
            a_re_stdev = np.std(a_recalls, ddof = 1)
            a_fm_stdev = np.std(a_fmeasures, ddof = 1)
            a_ee_stdev = np.std(a_eers, ddof = 1)
            a_ee_th_stdev = np.std(a_eer_thetas, ddof = 1)
            a_fa_stdev = np.std(a_fars, ddof = 1)
            a_fa_th_stdev = np.std(a_far_thetas, ddof = 1)
            
            a_auroc_stdev = np.std(a_aurocs, ddof = 1)
            a_losses_stdev = np.std(a_losses, ddof = 1)

            output[key]["fm"] = (get_average(a_fmeasures), a_fm_stdev)
            output[key]["prec"] = (get_average(a_precisions), a_pr_stdev)
            output[key]["rec"] = (get_average(a_recalls), a_re_stdev)
            output[key]["eer"] = (get_average(a_eers), a_ee_stdev)
            output[key]["far"] = (get_average(a_fars), a_fa_stdev)
            output[key]["auroc"] = (get_average(a_aurocs), a_auroc_stdev)
            output[key]["loss"] = (get_average(a_losses), a_losses_stdev)


            with open(f'data/stats/{save_name}.pickle', 'wb') as handle:
                pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

            
    # multiple_models_scaled_v2.pickle was a good summary of different types of model

    return output, (all_scores_legit, all_scores_adv)

def summarise_by_model(output, models=["WatchAuth"]):
    
    for stat in ["fm", "prec", "rec", "eer", "far", "auroc", "loss"]:
        for model_name in models:
            l = []
            for auth_user in range(16):
                l.append(output[f"{auth_user}_{model_name}"][stat][0])
            print(model_name, stat, sum(l) /16)
        print("")

In [38]:
from testing_results import run as run_normal

models = ["vae_gan_specific_50_gestures=2"]
model = models[0]


original_restricted = run_normal(models=["WatchAuth"], terminal_types = [3], median_filtering=True, score_mode="macro", limit_data_first_x=5, repetitions=3)



"""
outputs = {"original_restricted":original_restricted}
for model in models:
    for method in [ "just_sampled",  "adversarially_mixed", "self_mixed", "same_user_other_gestures_terminal=345"]: #, ]:   #["just_sampled", "adversarially_mixed", "self_mixed", "same_user_other_gestures_terminal=345"]: # 

        handle = lambda x: f"data/generated_samples/user={x}_model={model}_method={method}.npy"

        output = run(models=["WatchAuth"], median_filtering=True, score_mode="macro", extra_data_handle = handle, repetitions=3, save_name=f"{model}-{method}-macro-synth")

        outputs[f"{model}-{method}-macro"] = output  
"""

auth_user is 0
(array([0, 1]), array([2254,    5]))
auth_user is 1
(array([0, 1]), array([2262,    5]))
auth_user is 2
(array([0, 1]), array([2248,    5]))
auth_user is 3
(array([0, 1]), array([2279,    5]))
auth_user is 4
(array([0, 1]), array([2180,    5]))
auth_user is 5
(array([0, 1]), array([2248,    5]))
auth_user is 6
(array([0, 1]), array([2253,    5]))
auth_user is 7
(array([0, 1]), array([2245,    5]))
auth_user is 8
(array([0, 1]), array([2311,    5]))
auth_user is 9
(array([0, 1]), array([2314,    5]))
auth_user is 10
(array([0, 1]), array([2245,    5]))
auth_user is 11
(array([0, 1]), array([2211,    5]))
auth_user is 12
(array([0, 1]), array([2239,    5]))
auth_user is 13
(array([0, 1]), array([2236,    5]))
auth_user is 14
(array([0, 1]), array([2230,    5]))
auth_user is 15
(array([0, 1]), array([2230,    5]))


'\noutputs = {"original_restricted":original_restricted}\nfor model in models:\n    for method in [ "just_sampled",  "adversarially_mixed", "self_mixed", "same_user_other_gestures_terminal=345"]: #, ]:   #["just_sampled", "adversarially_mixed", "self_mixed", "same_user_other_gestures_terminal=345"]: # \n\n        handle = lambda x: f"data/generated_samples/user={x}_model={model}_method={method}.npy"\n\n        output = run(models=["WatchAuth"], median_filtering=True, score_mode="macro", extra_data_handle = handle, repetitions=3, save_name=f"{model}-{method}-macro-synth")\n\n        outputs[f"{model}-{method}-macro"] = output  \n'

In [39]:
from testing_results import summarise_by_model

model_type = ["WatchAuth"]

for key in outputs:
    print("original")

    summarise_by_model(original_restricted, model_type)
    if key != "original_restricted":
        print(key)
        summarise_by_model(outputs[key][0],model_type) #s[key]

    print("_"*50)
    

original


TypeError: tuple indices must be integers or slices, not str

In [25]:
output[1]

([0.23,
  0.23,
  0.12,
  0.17,
  0.15,
  0.1,
  0.28,
  0.2,
  0.2,
  0.15,
  0.12,
  0.2,
  0.21,
  0.12,
  0.15,
  0.14,
  0.14,
  0.1,
  0.18,
  0.55,
  0.24,
  0.17,
  0.28,
  0.27,
  0.28,
  0.1,
  0.15,
  0.45,
  0.24,
  0.24,
  0.19,
  0.13,
  0.15,
  0.09,
  0.06,
  0.05,
  0.21,
  0.18,
  0.23,
  0.2,
  0.17,
  0.25,
  0.16,
  0.29,
  0.13,
  0.14,
  0.08,
  0.1,
  0.26,
  0.13,
  0.17,
  0.14,
  0.13,
  0.17,
  0.15,
  0.14,
  0.14,
  0.19,
  0.41,
  0.13,
  0.2,
  0.27,
  0.11,
  0.18,
  0.22,
  0.21,
  0.17,
  0.13,
  0.15,
  0.06,
  0.35,
  0.14,
  0.15,
  0.16,
  0.15,
  0.19,
  0.2,
  0.16,
  0.13,
  0.14,
  0.14,
  0.18,
  0.19,
  0.58,
  0.26,
  0.15,
  0.24,
  0.25,
  0.22,
  0.08,
  0.15,
  0.54,
  0.31,
  0.24,
  0.21,
  0.21,
  0.09,
  0.11,
  0.12,
  0.1,
  0.22,
  0.14,
  0.27,
  0.24,
  0.24,
  0.18,
  0.23,
  0.29,
  0.19,
  0.1,
  0.14,
  0.1,
  0.21,
  0.16,
  0.12,
  0.18,
  0.15,
  0.07,
  0.19,
  0.2,
  0.14,
  0.24,
  0.24,
  0.19,
  0.2,
  0.16,
  0.19,

In [ ]:
scores_legit, scores_adv = original_restricted[1] #outputs["vae_gan_specific_50_gestures=2-adversarially_mixed-macro"][1]
eer = get_eer(scores_legit, scores_adv)
far = get_far_when_zero_frr(scores_legit, scores_adv)
display(sum(1 if e == 0 else 0 for e in scores_adv) / len(scores_adv))

frr = sum(1 if e == 0 else 0 for e in scores_legit) / len(scores_legit)

display(eer)
display(far)
display(frr)

## Other stuff

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, LSTM, Permute, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam

from keras.regularizers import l2, l1


from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
import sklearn

import pickle

from featurize import featurize
from visualise import *

from scaler import CustomScaler

from VAE_stats import VAE_stats

2023-02-12 13:18:05.633139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 13:18:06.612902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-12 13:18:06.613069: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-12 13:18:06.613069: W tensorfl

In [1]:
tf.get_logger().setLevel('ERROR')

device_name = tf.test.gpu_device_name()

file_name = "raw_with_maps" # or offsets_2

x_data = np.load(f"data/processed/x_{file_name}_median_filtered.npy")

#from featurize import filter as _filter, ORDER, CUTOFF, FILTER_INDICES
#x_data = _filter(x_data, FILTER_INDICES, ORDER, CUTOFF)

y_user = np.load(f"data/processed/y_user_{file_name}.npy")
y_intent = np.load(f"data/processed/y_intent_{file_name}.npy").astype(int)
y_gesture = np.load(f"data/processed/y_gesture_type_{file_name}.npy")

train_gesture_map = np.load(f"data/processed/train_gesture_map_{file_name}.npy").astype(int)
test_gesture_map = np.load(f"data/processed/test_gesture_map_{file_name}.npy").astype(int)

###


_map = (train_gesture_map == 1) #& ((y_user.argmax(axis=1) >= 11))

user_x = x_data[_map]
user_y = y_user.argmax(axis=1)[_map]

def show_visuals(vae):
    
    
    latent_space_means, latent_space_stds, latent_space = vae.encoder(vae.scaler.transform(x_data[:10]))

    plt.scatter(latent_space_means[:,0], latent_space_means[:,1], c = range(10), label="means")
    plt.scatter(latent_space[:,0], latent_space[:,1], label="sampled", c = range(10), s=10)

    plt.legend()
    
    plot_label_clusters(vae, vae.scaler.transform(user_x), user_y)
    visualise(vae, vae.scaler.transform(user_x), user_y, 10)
    plot_reconstructed_curves(vae, vae.scaler.transform(user_x), channel=0)


NameError: name 'tf' is not defined

In [11]:
vae = VAE_stats(50)

x_feature_array = []
print(vae.vae_featurize(x_data[0:1])[0].shape)

for i in range(0, len(x_data), 128):
    print(i)
    x_feature_array.append(vae.vae_featurize(x_data[i:i+128]))



(144,)
0
128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048
2176
2304
2432
2560
2688
2816
2944
3072
3200
3328
3456
3584
3712
3840
3968
4096
4224
4352
4480
4608
4736
4864
4992
5120
5248
5376
5504
5632
5760
5888
6016
6144
6272
6400
6528
6656
6784
6912
7040
7168
7296
7424
7552
7680
7808
7936
8064
8192
8320
8448
8576
8704
8832
8960
9088
9216
9344
9472
9600
9728
9856
9984
10112
10240
10368
10496
10624
10752
10880
11008
11136
11264
11392
11520
11648
11776
11904
12032
12160
12288
12416
12544
12672
12800
12928
13056
13184
13312
13440
13568
13696
13824
13952
14080
14208
14336
14464
14592
14720
14848
14976
15104
15232
15360
15488
15616
15744
15872
16000
16128
16256
16384
16512
16640
16768
16896
17024
17152
17280
17408
17536
17664
17792
17920
18048
18176
18304
18432
18560
18688
18816
18944
19072
19200
19328
19456
19584
19712
19840
19968
20096
20224
20352
20480
20608
20736
20864
20992
21120
21248
21376
21504
21632
21760
21888
22016
22144
22272
22400
22528
22656
22784
22912
23

In [15]:
display(x_feature_array[0].shape)
x_features = np.concatenate(x_feature_array, axis=0)
print(x_features.shape)
np.save(f"data/processed/x_{file_name}_median_filtered_features.npy", x_features)

TensorShape([128, 144])

(34255, 144)


In [ ]:
for latent_dim in [50]: # , 30, 10, 100]: # 500, 50, 40, 30, 20, 10

#latent_dim = 100
    """
    vae = VAE_stats(latent_dim)
    vae.fit_scaler(x_data[(y_intent==0)])
    vae.auth_on = False
    
    show_visuals(vae)

    initial_map = (train_gesture_map==1) | (y_intent == 0)
    train_data = vae.scaler.transform(x_data[initial_map])

    print("Fitting basic")

    vae.beta = 1e-4
    vae.compile(Adam())
    history_1 = vae.fit(train_data, y_user[initial_map], epochs=15, batch_size=128, verbose=1)
    vae.save_model(f"all_users", f"vae_stats_basic_reconstruction_{latent_dim}")
    """
    
    specific_map = (train_gesture_map==1) 
    vae = VAE_stats(latent_dim)
    vae.load_model(f"all_users", f"vae_stats_basic_reconstruction_{latent_dim}")

    show_visuals(vae)
    
    beta = 1e-4
    vae.beta = beta
    vae.alpha = 0.01
    vae.delta = 0.11
    vae.auth_on = True

    vae.compile(Adam())

    epochs = 5
    train_data = vae.scaler.transform(x_data[specific_map])

    for i in range(10):
        print("Fitting specific")

        vae.fit(train_data, y_user[specific_map], epochs=100, batch_size=128, verbose=1)

        show_visuals(vae)
    """
    beta = 1e-2
    vae.beta = beta
    vae.alpha = 0.5
    vae.auth_on = True
    print("Fitting specific - higher auth")
    
    vae.fit(train_data, y_user[specific_map], epochs=1000, batch_size=128, verbose=1)
    
    show_visuals(vae)
    """
    
    vae.save_model(f"all_users", f"vae_stats_specific_v2_{latent_dim}")

In [ ]:
def feature_max(g_data):
	return tf.math.reduce_max(g_data, axis=1)
 
def feature_min(g_data):
	return tf.math.reduce_min(g_data, axis=1)
 

def feature_mean(g_data):
	return tf.math.reduce_mean(g_data, axis=1)
    
def feature_stdev(g_data):
	return tf.math.reduce_std(g_data, axis=1)

def feature_var(g_data):
	return tf.math.reduce_variance(g_data, axis=1)

def feature_median(g_data):
    return tfp.stats.percentile(g_data, 50.0, interpolation='midpoint', axis=1)

def feature_iqr(g_data):
    return tfp.stats.percentile(g_data, 75.0, interpolation='midpoint', axis=1) - tfp.stats.percentile(g_data, 25.0, interpolation='midpoint', axis=1)

def feature_kurt(g_data):
    std = tf.math.reduce_std(g_data, axis=1)
    diff = tf.math.reduce_mean((g_data - tf.math.reduce_mean(g_data, axis=1, keepdims = True))**4, axis=1)
    return diff / std**4

def feature_skew(g_data):
    std = tf.math.reduce_std(g_data, axis=1)
    diff = tf.math.reduce_mean((g_data - tf.math.reduce_mean(g_data, axis=1, keepdims = True))**3, axis=1)
    return diff / std**3

In [79]:
file_name = "raw_with_maps" # or offsets_2

x_data = np.load(f"data/processed/x_{file_name}_filtered.npy")
y_user = np.load(f"data/processed/y_user_{file_name}.npy")
y_intent = np.load(f"data/processed/y_intent_{file_name}.npy").astype(int)
y_gesture = np.load(f"data/processed/y_gesture_type_{file_name}.npy")

train_gesture_map = np.load(f"data/processed/train_gesture_map_{file_name}.npy").astype(int)

test_gesture_map = np.load(f"data/processed/test_gesture_map_{file_name}.npy").astype(int)


In [80]:
trial_data = x_data[:128]

In [81]:
data = tf.convert_to_tensor(trial_data)

In [82]:
_max = feature_max(data)
_min = feature_min(data)
_mean = feature_mean(data)
_std = feature_stdev(data)
_var = feature_var(data)
_median = feature_median(data)
_iqr = feature_iqr(data)
_kurt = feature_kurt(data)
_skew = feature_skew(data)

features = tf.concat([_max, _min, _mean, _std, _var, _median, _iqr, _kurt, _skew], axis=1)
features

<tf.Tensor: shape=(128, 144), dtype=float64, numpy=
array([[ 1.17380573, -2.23082167,  9.58853584, ..., -0.381646  ,
         0.28685247,  0.36725875],
       [ 0.22855253, -2.97880558,  9.11028331, ..., -0.34530999,
        -0.44720132,  0.3219769 ],
       [ 0.71983189, -2.99275854,  9.2993504 , ...,  0.83732114,
        -1.17741003,  0.83818612],
       ...,
       [ 3.18385887, -2.67086618, 10.53634914, ..., -1.32756023,
        -0.24308826,  1.43226517],
       [ 0.35267441, -4.88235892,  8.34485913, ..., -1.86707896,
        -0.75049372,  0.55233989],
       [ 1.19715888, -0.99358881,  9.36356918, ...,  0.35623664,
        -1.63732195,  0.62811479]])>

In [69]:
display(trial_data)
display( tf.math.reduce_mean(trial_data, axis=1, keepdims = True))
display(trial_data  - tf.math.reduce_mean(trial_data, axis=1, keepdims = True))
display((trial_data  - tf.math.reduce_mean(trial_data, axis=1, keepdims = True))**4)
display(tf.math.reduce_mean((trial_data  - tf.math.reduce_mean(trial_data, axis=1, keepdims = True))**4, axis=1))
display(tf.math.reduce_mean((trial_data  - tf.math.reduce_mean(trial_data, axis=1, keepdims = True))**4, axis=1) / tf.math.reduce_std(trial_data, axis=1)**4 )

<tf.Tensor: shape=(128, 200, 16), dtype=float64, numpy=
array([[[ 1.17380573e+00, -2.58720799e+00,  9.58853584e+00, ...,
         -6.96151705e+00,  2.23089887e+00,  8.14994100e+00],
        [ 1.13823141e+00, -2.53876070e+00,  9.56722722e+00, ...,
         -6.83049185e+00,  2.55111246e+00,  8.10702500e+00],
        [ 1.10481621e+00, -2.49456617e+00,  9.54612329e+00, ...,
         -6.69454243e+00,  2.84177267e+00,  8.31343100e+00],
        ...,
        [-1.01573015e+00, -3.91179250e+00, -8.92585399e+00, ...,
          1.97838667e+00, -1.64349376e+00,  3.28856500e+00],
        [-1.01526334e+00, -3.92784447e+00, -8.91247754e+00, ...,
          1.93335272e+00, -1.58150781e+00,  3.27801900e+00],
        [-1.01411311e+00, -3.94542381e+00, -8.89980296e+00, ...,
          1.88489797e+00, -1.52064250e+00,  2.98492300e+00]],

       [[ 2.25204930e-02, -6.63771436e+00,  9.11028331e+00, ...,
         -3.09413823e+00, -3.00602563e-02,  3.10295400e+00],
        [ 4.74700950e-02, -6.27461778e+00,  8.8

<tf.Tensor: shape=(128, 1, 16), dtype=float64, numpy=
array([[[-0.10715186, -4.0124292 ,  1.65174318, ..., -0.36785356,
         -1.12441811,  4.17918539]],

       [[-0.49439555, -4.85515248,  0.13258025, ...,  0.79979367,
         -1.6168396 ,  3.31411813]],

       [[-0.53934345, -4.90105357,  3.9796861 , ...,  0.77064798,
         -1.05054327,  2.01857928]],

       ...,

       [[-0.94539079, -6.5275634 ,  3.96524293, ..., -0.81100671,
         -0.32052866,  3.02705698]],

       [[-0.95769577, -6.32157963,  3.84019919, ...,  0.17634873,
          0.41917022,  2.97261469]],

       [[-0.07504546, -8.18444418, -0.8867777 , ..., -0.96772822,
          0.32706953,  3.49886675]]])>

<tf.Tensor: shape=(128, 200, 16), dtype=float64, numpy=
array([[[  1.28095759,   1.4252212 ,   7.93679266, ...,  -6.59366349,
           3.35531698,   3.97075561],
        [  1.24538327,   1.4736685 ,   7.91548404, ...,  -6.46263829,
           3.67553056,   3.92783961],
        [  1.21196808,   1.51786303,   7.89438011, ...,  -6.32668886,
           3.96619077,   4.13424561],
        ...,
        [ -0.90857829,   0.1006367 , -10.57759717, ...,   2.34624024,
          -0.51907565,  -0.89062039],
        [ -0.90811147,   0.08458473, -10.56422072, ...,   2.30120629,
          -0.4570897 ,  -0.90116639],
        [ -0.90696125,   0.06700538, -10.55154614, ...,   2.25275153,
          -0.39622439,  -1.19426239]],

       [[  0.51691604,  -1.78256188,   8.97770306, ...,  -3.89393189,
           1.58677934,  -0.21116413],
        [  0.54186564,  -1.4194653 ,   8.74495788, ...,  -3.27403376,
           1.47043514,  -1.03356013],
        [  0.56633574,  -1.07719041,   8.52747938, ...,  -2.68958

<tf.Tensor: shape=(128, 200, 16), dtype=float64, numpy=
array([[[2.69239646e+00, 4.12599882e+00, 3.96807745e+03, ...,
         1.89019720e+03, 1.26745987e+02, 2.48595140e+02],
        [2.40553738e+00, 4.71627587e+00, 3.92563494e+03, ...,
         1.74437317e+03, 1.82507264e+02, 2.38020840e+02],
        [2.15756914e+00, 5.30799291e+00, 3.88393663e+03, ...,
         1.60216009e+03, 2.47453955e+02, 2.92136013e+02],
        ...,
        [6.81474197e-01, 1.02571218e-04, 1.25183790e+04, ...,
         3.03032991e+01, 7.25976608e-02, 6.29173649e-01],
        [6.80074755e-01, 5.11879629e-05, 1.24551759e+04, ...,
         2.80428538e+01, 4.36521619e-02, 6.59507796e-01],
        [6.76635732e-01, 2.01575980e-05, 1.23955103e+04, ...,
         2.57545031e+01, 2.46470435e-02, 2.03422513e+00]],

       [[7.13970124e-02, 1.00966770e+01, 6.49622335e+03, ...,
         2.29907641e+02, 6.33966264e+00, 1.98829376e-03],
        [8.62117505e-02, 4.05974845e+00, 5.84831677e+03, ...,
         1.14903329e+02, 4.

<tf.Tensor: shape=(128, 16), dtype=float64, numpy=
array([[7.21221660e-01, 4.91903634e+01, 6.59595874e+03, ...,
        1.54698191e+02, 2.71016193e+01, 9.90385308e+01],
       [9.56046101e-02, 2.29821287e+01, 5.44950296e+03, ...,
        3.91537215e+01, 1.71859048e+01, 2.77437351e+01],
       [4.58212441e-01, 2.03672813e+01, 6.58288794e+03, ...,
        1.28716550e+01, 3.63123347e+01, 4.11491769e+01],
       ...,
       [1.23878519e+01, 1.18536572e+01, 3.98311363e+03, ...,
        2.22072316e+02, 1.79348935e+01, 2.40220222e+02],
       [2.63659476e-01, 6.92254580e+00, 4.28730026e+03, ...,
        1.22905628e+02, 5.38968289e+01, 6.18389292e+01],
       [7.68882895e-01, 1.63069153e+02, 1.47250589e+03, ...,
        1.19841638e+02, 6.85062716e+01, 5.27896058e+01]])>

<tf.Tensor: shape=(128, 16), dtype=float64, numpy=
array([[1.201262  , 5.46334932, 1.1774653 , ..., 1.95322272, 2.56115095,
        2.71780486],
       [1.85331584, 5.72000602, 1.06268927, ..., 1.88840455, 2.59585253,
        1.96282621],
       [2.42153196, 9.26472397, 2.13121657, ..., 2.55454018, 3.22804511,
        1.95613293],
       ...,
       [5.61528187, 5.90874765, 2.81029872, ..., 4.21202296, 2.4787322 ,
        4.60330872],
       [2.12859411, 8.41481702, 2.60689201, ..., 6.60891902, 3.54179118,
        2.29145415],
       [2.10639996, 3.53126772, 2.63754659, ..., 3.2928732 , 6.40018638,
        3.09699744]])>